In [28]:
import requests, csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
import re
import tqdm
import json
import tqdm
from difflib import SequenceMatcher
from concurrent.futures import ThreadPoolExecutor, as_completed

In [12]:
head = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'fr;q=0.9,en-US,en;q=0.8',
      'Connection': 'keep-alive',
      'refere': 'https://imdb.com'}

In [17]:
def month_movie(year, month):
    
    month_dict = {'january':'01', 'february':'02', 'march':'03', 'april':'04', 'may':'05', 'june':'06', \
                'july':'07', 'august':'08', 'september':'09', 'october':'10', 'november':'11', 'december':'12'}
    
    req1 = Request(f"https://www.allocine.fr/film/agenda/mois/mois-{year}-{month_dict[month]}", headers=head)
    webpage1 = urlopen(req1).read()
    bs = BeautifulSoup(webpage1, "html.parser")
    movies = bs.findAll('a', {'class': "month-movies-link"})
    movie_list = [text.text.strip() for text in movies]

    return movie_list

In [18]:
month = month_movie(2026, 'january')

In [15]:
def def get_movies_multithread(queries, head_title, max_workers=5):
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(get_similar_movies, query, head_title.copy()): query
            for query in queries
        }

        for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
            query = futures[future]
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Erreur pour '{query}':", e)

    return pd.concat(results, ignore_index=True)(query: str, head_title: dict[str,str]) -> pd.DataFrame:

    query_dict = {"\xa0": "","\x9c": "oe"," !": "!"," ?": "?"," :": ":","’": "'","–": "-","\x92": "'","\xc2":"â",\
                  "\xe1":"a","\xf1":"n","\xf6":"o","\x96":"-","\x8c":"oe","\xae":"","ì":"i","\xfc":"u","\xdc":"U"}
    
    # dictionary of characters to change in URL
    char_dict = {" ": "%20","!": "%21", "#": "%23", "$": "%24", "&": "%26", "'": "%27", "(": "%28", ")": "%29", \
                 "*": "%2A", "+": "%2B", ",": "%2C", "/": "%2F", ":": "%3A", ";": "%3B", "=": "%3D", "?": "%3F", \
                 "@": "%40", "[": "%5B", "]": "%5D", "â": "a", "à": "a", "é": "e", "è": "e", "ê": "e", "ë": "e", \
                 "ç": "c", "ï": "i", "ô": "o", "î": "i", "’": "%27", "ó":"o", "ł":"l", "ù":"u","Î":"i","ú":"u", \
                "'":"%27","À":"a","·":".","°":"","č":"c","É":"e","œ":"oe","û":"u","Ç":"c"}
    
    query = query.strip()
    for old, new in query_dict.items():
        query = query.replace(old, new)

    modif_query = [char_dict[i] if i in char_dict.keys() else i for i in query]
    modif_query = "".join(modif_query) + "&ref_=nv_sr_sm"
        
    head_title['Accept-Language'] = 'fr;q=0.9'    

    req1 = Request(url="https://www.imdb.com/find/?s=tt&q="+modif_query, headers=head_title)
    webpage1 = urlopen(req1).read()
    bs = BeautifulSoup(webpage1, "html.parser")
    
    items = bs.select('li[class*="ipc-metadata-list-summary-item"]')[:6]  # 6 premiers résultats

    results = []
    for li_tag in items:
        # Titre
        title_tag = li_tag.select_one("h3.ipc-title__text")
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)

        # Code IMDb
        link_tag = li_tag.select_one("a[href*='/title/tt']")
        if not link_tag:
            continue
        imdb_id = link_tag["href"].split("/title/")[1].split("/")[0]

        # year
        try: 
            text = li_tag.select("span")[1].text

            match = re.search(r"(19|20)\d{2}", text)
            year = int(match.group()) if match else 1492
        except ValueError:
            year = 1492

        # Similarity
        similarity = SequenceMatcher(None, query.lower(), title.lower()).ratio()

        if similarity >= 0.6 and year > 2022:
            results.append({
                "imdb_id": imdb_id,
                "query": query,
                "title": title,
                "year": year,
                "similarity": round(similarity, 3)
            })

        if len(results) == 0:
            results = [{"title": query, "imdb_id": "", "year": "", "similarity": ""}]

    return pd.DataFrame(results)

In [25]:
def get_movies_multithread(queries, head_title, max_workers=10):
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(get_similar_movies, query, head_title.copy()): query
            for query in queries
        }

        for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
            query = futures[future]
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Erreur pour '{query}':", e)

    return pd.concat(results, ignore_index=True)

In [26]:
new_df = get_movies_multithread(month, head, max_workers=10)

100%|██████████████████████████████████████████| 71/71 [00:11<00:00,  6.39it/s]


In [29]:
new_df.to_csv("new_movies.csv", encoding="ISO-8859-1", index=False, sep=";")